# Imports

In [23]:
# Used to pull data from Reddit
import praw
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns

from gensim.models import Word2Vec
from nltk.corpus import brown
import nltk
import tensorflow as tf
import tensorflow_hub as hub
import multiprocessing
from google.cloud import bigquery

#nltk.download('punkt')
#nltk.download('brown')

In [5]:
sql_query = '''
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2018_10` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2018_11` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2018_12` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_01` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_02` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.body
FROM 
  `fh-bigquery.reddit_comments.2019_03` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
'''

# Put query results into df
comment_df = pd.read_gbq(sql_query,
                         project_id='w266-240122',
                         dialect='standard')

# Convert date into proper date/time
comment_df['created_dt_tm'] = comment_df['created_utc'].apply(lambda x: dt.datetime.fromtimestamp(x))

# Create field for month
comment_df['created_dt_month'] = comment_df['created_dt_tm'].dt.to_period('M').dt.to_timestamp()

In [6]:
sql_query = """
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2018_10` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2018_11` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2018_12` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_01` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_02` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
UNION ALL
SELECT 
    r.subreddit,
    r.created_utc,
    r.title,
    r.selftext,
    r.is_self
FROM 
  `fh-bigquery.reddit_posts.2019_03` r
WHERE
  r.subreddit IN ('democrats', 'progressive', 'Conservative', 'republicans')
"""

post_df = pd.read_gbq(sql_query,
                      project_id='w266-240122',
                      dialect='standard')

# Convert date into proper date/time
post_df['created_dt_tm'] = post_df['created_utc'].apply(lambda x: dt.datetime.fromtimestamp(x))

# Create field for month
post_df['created_dt_month'] = post_df['created_dt_tm'].dt.to_period('M').dt.to_timestamp()

# Functions

In [33]:
def preprocess_text(comment_df, post_df, subreddit):
    
    # TODO remove symbols, contractions
    # TODO lowercase words?
    # TODO Treat numbers as something else
    # Add beginning and end of sentence?
    # TODO how do we deal with stop words?
    
    # Body text from post dataframe
    post_text = list(post_df[(post_df['is_self'] == True) &
                             (~post_df['selftext'].isin(['[removed]', '[deleted]'])) &
                             post_df['subreddit'].isin(subreddit)]['selftext'].unique())
    
    # Add in text from title
    post_text += list(post_df[post_df['is_self'] == True]['title'].unique())
    
    # Add in text from post comments
    post_text += list(comment_df[(comment_df['subreddit'].isin(subreddit)) & 
                                     (~comment_df['body'].isin(['[removed]', '[deleted]']))]['body'])
    
    # Put all text into dataframe and drop dupes
    text_df = pd.DataFrame(post_text, columns=['text'])
    text_df.drop_duplicates(inplace=True)
    
    # Tokenize at sentence level
    text_df['sent_tokenized'] = text_df['text'].apply(nltk.sent_tokenize)
    text_df['sent_count'] = text_df['sent_tokenized'].apply(lambda x: len(x))

    # Put tokenized body into a list
    sent_list = list(text_df[text_df['sent_count'] > 0]['sent_tokenized'].apply(pd.Series).stack().unique())

    # Put list into dataframe
    sent_df = pd.DataFrame(sent_list, columns=['sentence'])

    # Tokenize each sentence at the word level
    sent_df['word_token'] = sent_df['sentence'].apply(nltk.word_tokenize)
    
    return sent_df

In [32]:
def train_embedding(sent_df):
    
    # train word embedding
    embedding = Word2Vec(list(sent_df['word_token']),
                         size=100,
                         window=5, 
                         min_count=5, 
                         negative=15, 
                         iter=10, workers=6)
    
    return embedding

# Basic EDA

In [10]:
comment_df['created_dt_month'].value_counts()

2019-03-01    109492
2018-10-01    107297
2019-01-01    100792
2018-11-01     91601
2019-02-01     88902
2018-12-01     73725
2018-09-01       765
Name: created_dt_month, dtype: int64

In [11]:
post_df['created_dt_month'].value_counts()

2018-10-01    11185
2019-01-01    10551
2019-03-01    10473
2018-11-01     9703
2019-02-01     9316
2018-12-01     7986
2018-09-01       94
Name: created_dt_month, dtype: int64

In [ ]:
sent_df = preprocess_text(comment_df, 
                          post_df,
                          ['democrats', 'progressive'])

In [19]:
sent_df['sent_length'] = sent_df['word_token'].apply(lambda x: len(x))

In [25]:
sent_df['sent_length'].describe()

count    162849.000000
mean         16.033215
std          12.799296
min           1.000000
25%           8.000000
50%          13.000000
75%          21.000000
max        1079.000000
Name: sent_length, dtype: float64

# Emebdding from Democratic Leaning Subreddits

### Text PreProcessing

In [ ]:
# Empty list to store the embeddings
embedding_list = []

# Dates to iterate over
dates = comment_df.created_dt_month.sort_values().unique()[1:]

# Iterate for each month and create embedding for it
for d in dates:
    sent_df = preprocess_text(comment_df[comment_df['created_dt_month'] == d], 
                              post_df[post_df['created_dt_month'] == d],
                             ['democrats', 'progressive'])
    word_embedding = train_embedding(sent_df)
    embedding_list.append(word_embedding)

In [129]:
embedding_list[0].wv.get_vector('Trump')

array([-1.7190933e+00,  4.1021172e-02,  1.1593300e+00,  2.0203650e+00,
        1.9874138e+00,  1.9087055e-03, -1.0063719e+00, -1.8183178e+00,
       -1.3092844e-01,  3.8338795e+00, -8.6535853e-01,  3.5451894e+00,
       -2.8931671e-01, -1.1620345e+00,  9.1748387e-01, -4.0269172e-01,
       -2.0906296e-01,  8.7279648e-02,  1.9909885e+00,  7.7803499e-01,
       -3.5342729e+00, -1.0143763e+00,  5.3909101e-02,  1.2877710e+00,
       -6.4435524e-01,  2.3384080e+00,  9.7574785e-02, -3.1455305e-01,
        3.8596442e-01,  1.0303618e+00, -1.4870282e-01,  1.5712183e+00,
        1.3617808e-01,  3.6301854e-01, -1.6417710e+00,  2.4938097e+00,
       -4.3287870e-01, -1.9931700e+00,  2.6318507e+00, -2.3353438e+00,
       -2.7420471e+00, -3.6454326e-01, -2.1236794e+00, -1.6058459e+00,
        4.0929160e-01,  8.8338435e-01, -4.9283695e-01,  3.2288483e-01,
        4.5569095e-01, -1.9589363e+00, -6.0223770e-01,  2.9102414e+00,
       -3.3178275e+00,  1.6971275e+00, -1.3805312e+00,  4.1557971e-01,
      

In [130]:
embedding_list[1].wv.get_vector('Trump')

array([-0.5075389 ,  0.14912793,  0.07176501,  1.1566166 ,  1.7253845 ,
        2.61282   , -1.4427612 , -0.99972963,  2.1480021 ,  0.8256554 ,
       -1.1864061 ,  2.311868  ,  0.7749732 ,  0.49654955,  0.20799069,
       -1.2363505 ,  0.10987767,  0.6288812 ,  1.3055888 ,  2.6335826 ,
       -0.95019555, -1.0392123 , -0.9106089 ,  1.8107651 , -2.1569471 ,
        3.4074106 , -1.6975914 , -1.8802112 ,  1.2819365 , -0.08315355,
       -1.8880098 ,  0.7893961 ,  1.5719092 ,  1.4571885 , -2.4410691 ,
        0.6761529 , -1.2647302 , -0.31568453,  1.7645837 , -3.632652  ,
       -2.0368884 ,  1.3229909 , -1.9676155 ,  0.50106   ,  0.01585384,
        1.7202361 , -1.7338029 ,  1.3675741 , -0.0046582 ,  1.4361967 ,
        0.935738  ,  1.9071585 , -2.9139009 ,  1.5232185 , -1.0181456 ,
       -1.2355169 ,  3.353508  , -2.7842114 , -0.8736882 ,  1.7795514 ,
       -1.4942611 , -0.46974042,  1.2316047 , -0.08335562,  2.190173  ,
       -3.920645  , -1.6090406 , -1.1650267 , -0.24947233, -0.69

### ELMo

In [20]:
# Download ELMo to a local directory
!mkdir module_elmo

# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module_elmo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  331M    0   415    0     0    369      0  10d 21h  0:00:01  10d 21h   369x assets/
x saved_model.pb
x tfhub_module.pb
x variables/
x variables/variables.index
100  331M  100  331M    0     0  2054k      0  0:02:45  0:02:45 --:--:-- 2379k 2614kk      0  0:02:59  0:01:26  0:01:33 2755k   0  2087k      0  0:02:42  0:01:43  0:00:59 3369k      0  0:02:41  0:01:51  0:00:50 2300k  0:01:56  0:00:44 2391k0:02:44  0:02:25  0:00:19 2082k



In [4]:
# Initialize elmo
elmo = hub.Module('module_elmo', trainable=True)
embeddings = elmo(['Trump'], signature="default", as_dict=True)['elmo']

# Get word embedding
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

Instructions for updating:
Colocations handled automatically by placer.


W0715 22:15:39.981253 4808177088 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0715 22:15:40.683024 4808177088 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [11]:
message_embeddings[0][0]

1024

In [5]:
# Download ELMo to a local directory
!mkdir module_elmo

# Download the module, and uncom`press it to the destination folder. 
!curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module_elmo

# Initialize elmo
elmo = hub.Module('module_elmo', trainable=True)
embeddings = elmo(['This is an example sentence that I want an embedding for'], signature="default", as_dict=True)['elmo']

# Get word embedding
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

message_embeddings.shape

(1, 11, 1024)

# Questions
* Sentiment analysis labeled data
* How would we go about scoring the performance of our models?
    * I proposed to use the scores that are available on the reddit
* Should we train the sentiment classifier at a sentence or post level?
    * Each post may contain multiple sentences


### Data from r/Republican Subreddit

In [91]:
len(list(dem_posts['sent_tokenized_title'].apply(pd.Series).stack()))

1497

In [92]:
1497 + 714

2211

In [90]:
len(test)

2211

In [61]:
republicans = reddit.subreddit('Republican')

In [76]:
posts = []
# for post in democrats.top(limit = 1):
for post in republicans.hot(limit = 1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created, post.is_self, post.link_flair_text])
rep_posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created', 'is_self', 'link_flair_text'])
created = rep_posts["created"].apply(lambda x: dt.datetime.fromtimestamp(x))
rep_posts['created'] = created